# AceleraDev DataScience

## Setup

kaggle: House Prices Advanced regression techniques

## ML Canvas

### Qual é a proposta de valor?
O que é esperado do modelo, para que ele servirá, para quem servirá.

Ex: Prever o preço do imóvel com base nas variáveis x,y,z

### Para que tipo de decisões o modelo será usado?

Será necessário calcular de quanto em quanto tempo?

### Quais são minhas fontes de dados?

- Que tipos de fontes?
- São internas? Externas?
- Nacional, Internacional?
- Como os dados serão coletados?
- Qual a complexidade de obter os dados?

As fontes não precisar ser fixas, é possível incluir remover fontes no decorrer do desenvolvimento, mas é necessário fazer uma discussão inicial

### Discussão de modelos

- Levanta mento de modelos, métodos, técnicas.

### Avaliações prévias

Métodos e métricas para avaliar o sistema antes dele ser empregado.

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("train.csv")

In [ ]:
df.columns

In [ ]:
df.isna().sum() 